In [1]:
import os
import requests
import json
import tensorflow as tf
import tensorflow_hub as hub
from pinecone import Pinecone, ServerlessSpec

In [3]:
pc = Pinecone(api_key='')


In [ ]:
pc.delete_index("rag")
pc.create_index(
    name="rag", dimension=512,metric="cosine",spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [21]:
data = json.load(open("reviews.json"))

In [9]:
data

{'reviews': [{'book': 'The Name of the Wind',
   'genre': 'Fantasy',
   'stars': 5,
   'review': 'A beautifully written, gripping story of a young wizard and his rise to legend.'},
  {'book': 'The Way of Kings',
   'genre': 'Fantasy',
   'stars': 5,
   'review': 'An epic journey filled with rich characters, political intrigue, and magical landscapes.'},
  {'book': 'A Song of Ice and Fire',
   'genre': 'Fantasy',
   'stars': 4,
   'review': 'Complex characters and unpredictable plots in a brutally realistic world.'},
  {'book': 'The Hobbit',
   'genre': 'Fantasy',
   'stars': 5,
   'review': 'A classic adventure that set the stage for modern fantasy literature.'},
  {'book': 'The Lies of Locke Lamora',
   'genre': 'Fantasy',
   'stars': 4,
   'review': 'A thrilling heist story with rich world-building and witty banter.'},
  {'book': 'The Blade Itself',
   'genre': 'Fantasy',
   'stars': 4,
   'review': 'A gritty, realistic take on the fantasy genre with dark characters and sharp dialogu

In [23]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

processed_data = []

for review in data['reviews']:
    # Create embedding using Universal Sentence Encoder
    embedding = embed([review['review']])[0].numpy().tolist()

    # Append processed data
    processed_data.append({
        "values": embedding,
        "id": review['book'],
        "metadata": {
            "review": review['review'],
            "genre": review['genre'],
            "stars": review['stars']
        }
    })




In [24]:
processed_data

[{'values': [0.010670083574950695,
   0.05092041194438934,
   0.061778802424669266,
   -0.049194857478141785,
   -0.08263997733592987,
   0.0012193773873150349,
   0.037921685725450516,
   0.015819501131772995,
   0.06691864877939224,
   0.04721087962388992,
   0.05160154029726982,
   -0.08601115643978119,
   -0.03439074009656906,
   -0.019698813557624817,
   -0.05186663940548897,
   -0.07501783221960068,
   -0.029404882341623306,
   0.037292592227458954,
   0.013756868429481983,
   -0.021329760551452637,
   0.04468992352485657,
   0.0542852021753788,
   -0.005229142494499683,
   0.06832828372716904,
   0.036639321595430374,
   -0.04305644705891609,
   -0.04064916446805,
   -0.03354114666581154,
   0.030552461743354797,
   -0.029377607628703117,
   -0.003934834618121386,
   -0.016690881922841072,
   0.03178383782505989,
   0.06874189525842667,
   0.008367846719920635,
   0.03759157657623291,
   -0.06250391900539398,
   0.03791254758834839,
   0.009333260357379913,
   -0.007638803683221

In [25]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 90}

In [29]:
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 89}},
 'total_vector_count': 89}

In [31]:
!pip install pip-chill

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
